In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms


In [2]:
custom_transform= transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

In [3]:
train_set=datasets.FashionMNIST('./data', train=True, download=True, transform=custom_transform)

In [4]:
train_set

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )

In [5]:
def get_data_loader(training=True):
    custom_transform= transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
    
    if training == True:
        train_set=datasets.FashionMNIST('./data', train=training, download=True, transform=custom_transform)
        train_loader = torch.utils.data.DataLoader(train_set, batch_size = 64)
        return train_loader
    else:
        test_set=datasets.FashionMNIST('./data', train=False, download=True,transform=custom_transform)
        test_loader = torch.utils.data.DataLoader(test_set, batch_size = 64, shuffle=False)
        return test_loader
    

In [6]:
train_loader = get_data_loader()
print(type(train_loader))

<class 'torch.utils.data.dataloader.DataLoader'>


In [7]:
print(train_loader.dataset)

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )


In [8]:
test_loader = get_data_loader(False)
print(type(test_loader))
print(test_loader.dataset)

<class 'torch.utils.data.dataloader.DataLoader'>
Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )


In [9]:
def build_model():
    untrained_model = nn.Sequential(nn.Flatten(),
              nn.Linear(in_features= 28*28, out_features=120),
              nn.ReLU(),
             nn.Linear(in_features=120, out_features=60),
              nn.ReLU(),
             nn.Linear(in_features=60, out_features=10))
    return untrained_model

In [10]:
model = build_model()

In [11]:
print(model)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=784, out_features=120, bias=True)
  (2): ReLU()
  (3): Linear(in_features=120, out_features=60, bias=True)
  (4): ReLU()
  (5): Linear(in_features=60, out_features=10, bias=True)
)


In [12]:
criterion = nn.CrossEntropyLoss()

In [13]:
def train_model(model, train_loader, criterion, T):
    model.train()
    opt = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
    n = len(train_loader.dataset)
    for epoch in range(T):
        running_loss = 0.0
        correct = 0
        for i, data in enumerate(train_loader, 0):
            inputs, labels = data
            opt.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            opt.step()

            running_loss += loss.item()*50

            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
        print(f'Train Epoch: {epoch}\tAccuracy: {correct}/{n}({correct/n*100:.2f}%)\tLoss: {running_loss/n:.3f}')

In [14]:
train_model(model, train_loader, criterion, 5)

Train Epoch: 0	Accuracy: 42249/60000(70.42%)	Loss: 0.710
Train Epoch: 1	Accuracy: 49284/60000(82.14%)	Loss: 0.399
Train Epoch: 2	Accuracy: 50406/60000(84.01%)	Loss: 0.355
Train Epoch: 3	Accuracy: 51048/60000(85.08%)	Loss: 0.331
Train Epoch: 4	Accuracy: 51519/60000(85.87%)	Loss: 0.314


In [15]:
def evaluate_model(model, test_loader, criterion, show_loss = True):
    model.eval()
    n = len(test_loader.dataset)
    running_loss = 0.0
    correct = 0
    with torch.no_grad():
        
        for data, labels in test_loader:
            outputs = model(data)
            loss = criterion(outputs, labels)
            running_loss += loss.item()*50
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
    if not show_loss:
        print(f'Accuracy: {(correct/n)*100:.2f}%')
    else:
        print(f'Average loss: {(running_loss/n):.4f}')
        print(f'Accuracy: {(correct/n)*100:.2f}%')


In [16]:
evaluate_model(model, test_loader, criterion, show_loss = True)

Average loss: 0.3378
Accuracy: 84.70%


In [19]:
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot']

In [21]:
def predict_label(model, test_images, index):
    """
    TODO: implement this function.

    INPUT: 
        model - the trained model
        test_images   -  test image set of shape Nx1x28x28
        index   -  specific index  i of the image to be tested: 0 <= i <= N - 1


    RETURNS:
        None
    """
    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot']
    logits = model(test_images)
    prob = F.softmax(logits,dim=1)
    probs,labels = torch.topk(prob[index],3)
    
    for i in range(3):
        print(f'{class_names[labels[i]]}: {probs[i]*100:.2f}%')
    


In [20]:
?torch.topk

In [25]:
x = test_loader.dataset[0][0]
x = x[None]

In [27]:
x = torch.cat((x, (test_loader.dataset[1][0])[None]),0)

In [28]:
x

tensor([[[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]],


        [[[-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          ...,
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242],
          [-0.4242, -0.4242, -0.4242,  ..., -0.4242, -0.4242, -0.4242]]]])

In [30]:
predict_label(model, x, 1)

Pullover: 87.71%
Shirt: 10.96%
Coat: 1.21%


In [35]:
if __name__ == '__main__':
    train_loader = get_data_loader()
    model = build_model()
    criterion = nn.CrossEntropyLoss()
    test_loader = get_data_loader(False)
    test_images = test_loader.dataset[0][0]
    test_images = test_images[None]
    test_images = torch.cat((x, (test_loader.dataset[1][0])[None]),0)
    
    train_model(model, train_loader, criterion, 5)
    print()
    evaluate_model(model, test_loader, criterion, True)
    print()
    evaluate_model(model, test_loader, criterion, False)
    print()
    predict_label(model, test_images, 1)

Train Epoch: 0	Accuracy: 41750/60000(69.58%)	Loss: 0.722
Train Epoch: 1	Accuracy: 49111/60000(81.85%)	Loss: 0.406
Train Epoch: 2	Accuracy: 50356/60000(83.93%)	Loss: 0.358
Train Epoch: 3	Accuracy: 51046/60000(85.08%)	Loss: 0.333
Train Epoch: 4	Accuracy: 51513/60000(85.86%)	Loss: 0.315

Average loss: 0.3378
Accuracy: 84.73%

Accuracy: 84.73%

Pullover: 93.06%
Shirt: 6.19%
Coat: 0.68%
